<a href="https://colab.research.google.com/github/rynwhd/Language-Models/blob/main/Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Text Summarization using Vector Methods

In [ ]:
# Split the documents into sentences
# Score each sentence 
# Rank each sentence by those scores
# Summary=top scoring sentences

In [ ]:
#nltk.sent_tokenize(your_text)

In [ ]:
# Tf_IDF will give sentence x terms

In [1]:
# score=average(non-zero TF-IDF values)

In [6]:
# https://www.kaggle.com/shivamkushwaha/bbc-full-text-document-classification
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv


File ‘bbc_text_cls.csv’ already there; not retrieving.



In [39]:
import pandas as pd
import numpy as np
import nltk
import string
nltk.download(['punkt','stopwords'])
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import textwrap
import regex as re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
df=pd.read_csv('bbc_text_cls.csv')

In [9]:
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [10]:
doc=df[df.labels=='business']['text'].sample(random_state=42)

In [11]:
def wrap(x):
  return textwrap.fill(x,replace_whitespace=False,fix_sentence_endings=True)

In [12]:
print(wrap(doc.iloc[0]))

Christmas sales worst since 1981

UK retail sales fell in December,
failing to meet expectations and making it by some counts the worst
Christmas since 1981.

Retail sales dropped by 1% on the month in
December, after a 0.6% rise in November, the Office for National
Statistics (ONS) said.  The ONS revised the annual 2004 rate of growth
down from the 5.9% estimated in November to 3.2%. A number of
retailers have already reported poor figures for December.  Clothing
retailers and non-specialist stores were the worst hit with only
internet retailers showing any significant growth, according to the
ONS.

The last time retailers endured a tougher Christmas was 23 years
previously, when sales plunged 1.7%.

The ONS echoed an earlier
caution from Bank of England governor Mervyn King not to read too much
into the poor December figures.  Some analysts put a positive gloss on
the figures, pointing out that the non-seasonally-adjusted figures
showed a performance comparable with 2003. The Novembe

In [13]:
sents=doc.iloc[0].split('\n\n')[1:]

In [14]:
sents

['UK retail sales fell in December, failing to meet expectations and making it by some counts the worst Christmas since 1981.',
 'Retail sales dropped by 1% on the month in December, after a 0.6% rise in November, the Office for National Statistics (ONS) said. The ONS revised the annual 2004 rate of growth down from the 5.9% estimated in November to 3.2%. A number of retailers have already reported poor figures for December. Clothing retailers and non-specialist stores were the worst hit with only internet retailers showing any significant growth, according to the ONS.',
 'The last time retailers endured a tougher Christmas was 23 years previously, when sales plunged 1.7%.',
 'The ONS echoed an earlier caution from Bank of England governor Mervyn King not to read too much into the poor December figures. Some analysts put a positive gloss on the figures, pointing out that the non-seasonally-adjusted figures showed a performance comparable with 2003. The November-December jump last year 

In [15]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [16]:
text1='uk a, is , th! @ %'
re.sub(r"""[!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]""", ' ', text1)

'uk a  is   th     '

In [17]:
sent=[]
for text in sents:
  text=text.lower()
  text=re.sub(r"""[!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]""", ' ', text)
  sent.append(' '.join([words for words in text.split(' ') if not any(c.isdigit() for c in words)]))

In [18]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [19]:
vectorizer=TfidfVectorizer(stop_words=stopwords.words('english'))

In [20]:
X=vectorizer.fit_transform(sent)

In [21]:
X.shape

(6, 148)

In [22]:
score_sent_mapping={}
for i in range(X.shape[0]):
  score=X[i][X[i]!=0]
  score_sent_mapping[score.mean()]=i

In [24]:
score_sent_mapping

{0.2720637797748839: 0,
 0.16385460484606737: 1,
 0.3087308694363699: 2,
 0.13464752379333275: 3,
 0.2215540826117064: 4,
 0.1266002376182469: 5}

In [ ]:
#Many ways how to rank the sentences
# 1) Top N sentences
# 2) Top N words or characters
# 3) Top X% sentences or top X% words
# 4) Sentences with scores>average score
# 5) sentences with scores>factor*average_score

In [58]:
from collections import OrderedDict

In [63]:
score_sent_mapping=OrderedDict(sorted(score_sent_mapping.items()))

In [60]:
score_sent_mapping

OrderedDict([(0.1266002376182469, 5),
             (0.13464752379333275, 3),
             (0.16385460484606737, 1),
             (0.2215540826117064, 4),
             (0.2720637797748839, 0),
             (0.3087308694363699, 2)])

In [65]:
len(score_sent_mapping)

6

In [71]:
doc.iloc[0].split('\n\n')[0]

'Christmas sales worst since 1981'

In [70]:
count=0
for i,v in score_sent_mapping.items():
  if count>=len(score_sent_mapping)-3:
    print(wrap(sent[v]))
    print('....')
  count+=1

however  reports from some high street retailers highlight the
weakness of the sector  morrisons  woolworths  house of fraser  marks
spencer and big food all said that the festive period was
disappointing
....
uk retail sales fell in december  failing to meet expectations and
making it by some counts the worst christmas since
....
the last time retailers endured a tougher christmas was years
previously  when sales plunged
....


In [23]:
# for i,v in score_sent_mapping.items():
#   print(wrap(sent[v]))
#   print('.....')

In [27]:
def get_sent_score(s):
  return s[s!=0].mean()

In [48]:
def summarizer(text):
  sents=doc.split('\n\n')[1:]
  X=vectorizer.fit_transform(sents)
  scores=np.zeros(len(sents))
  for i in range(len(sents)):
    score=get_sent_score(X[i,:])
    scores[i]=score
  sort_idx=np.argsort(-scores)
  print(sort_idx)
  for i in sort_idx[:5]:
    print(wrap("%.2f : %s")%(scores[i],sents[i]))

In [55]:
doc=df[df.labels=='entertainment']['text'].sample(random_state=123).iloc[0]

In [53]:
doc

'Row threatens Hendrix museum plan\n\nProposals to open a museum dedicated to Jimi Hendrix are flailing because of a row over the home of his late father.\n\nThe run-down house in Seattle has already been moved wholesale once and local authorities are now demanding it be moved to another site. Hendrix supporters hoped to turn the home into a museum for the guitarist. "The mayor is going to go down as the mayor who destroyed Jimi Hendrix\'s house," said Ray Rae Marshall of the James Marshall Hendrix Foundation. The foundation moved the building, in which Al Hendrix lived between 1953 and 1956, when the land it was built on was to be developed for housing in 2002. Now the City of Seattle wants its new plot to be used for development, giving a deadline of 22 February for the home to be moved. Mr Goldman said the authority had promised the house could remain on its new site and be turned into a memorial and community centre. Seattle officials said no such deal had been offered.\n\n"We neve

In [56]:
summarizer(doc)

[0 1]
0.26 : Pop singer Delta Goodrem has scooped one of the top individual prizes at the first Australian MTV Music Awards.
0.08 : The 21-year-old singer won the award for best female artist, with Australian Idol runner-up Shannon Noll taking the title of best male at the ceremony. Goodrem, known in both Britain and Australia for her role as Nina Tucker in TV soap Neighbours, also performed a duet with boyfriend Brian McFadden. Other winners included Green Day, voted best group, and the Black Eyed Peas. Goodrem, Green Day and the Black Eyed Peas took home two awards each. As well as best female, Goodrem also took home the Pepsi Viewers Choice Award, whilst Green Day bagged the prize for best rock video for American Idiot. The Black Eyed Peas won awards for best R 'n' B video and sexiest video, both for Hey Mama. Local singer and songwriter Missy Higgins took the title of breakthrough artist of the year, with Australian Idol winner Guy Sebastian taking the honours for best pop video. T